In [2]:
import numpy as np
import pandas as pd

Есть ли связь между неграмотностью и рождаемостью? Для 94 стран, уровень неграмотности женщин в которых больше 5%, известны доля неграмотных среди женщин старше 15 (на 2003 год) и средняя рождаемость на одну женщину (на 2005 год). Чему равен выборочный коэффициент корреляции Пирсона между этими двумя признаками? Округлите до четырёх знаков после десятичной точки. 

In [2]:
data = pd.read_csv('illiteracy.txt', delimiter='\t')

In [3]:
data.head()

,Country,Illit,Births
0,Albania,20.5,1.78
1,Algeria,39.1,2.44
2,Bahrain,15.0,2.34
3,Belize,5.9,2.97
4,Benin,73.5,5.60


In [6]:
correlation = data.corr()

In [7]:
correlation

,Illit,Births
Illit,1.000000,0.768663
Births,0.768663,1.000000


In [8]:
correlation_spearman = data.corr('spearman')

In [9]:
correlation_spearman

,Illit,Births
Illit,1.000000,0.752962
Births,0.752962,1.000000


Для 61 большого города в Англии и Уэльсе известны средняя годовая смертность на 100000 населения (по данным 1958–1964) и концентрация кальция в питьевой воде (в частях на маллион). Чем выше концентрация кальция, тем жёстче вода. Города дополнительно поделены на северные и южные.

Есть ли связь между жёсткостью воды и средней годовой смертностью? Посчитайте значение коэффициента корреляции Пирсона между этими признаками, округлите его до четырёх знаков после десятичной точки.

In [10]:
water = pd.read_csv('water.txt', delimiter='\t')

In [11]:
water.head()

,location,town,mortality,hardness
0,South,Bath,1247,105
1,North,Birkenhead,1668,17
2,South,Birmingham,1466,5
3,North,Blackburn,1800,14
4,North,Blackpool,1609,18


In [13]:
print(f'Water correlation:\n {water.corr()}')

Water correlation:
            mortality  hardness
mortality   1.000000 -0.654849
hardness   -0.654849  1.000000


In [16]:
corr_spearman = water.corr('spearman')
print(f'Water Spearman correlation:\n {corr_spearman}')

Water Spearman correlation:
            mortality  hardness
mortality   1.000000 -0.631665
hardness   -0.631665  1.000000


Pearson correlation for North cities:

In [17]:
water[water['location'] == 'North'].corr()

,mortality,hardness
mortality,1.000000,-0.368598
hardness,-0.368598,1.000000


Pearson correlation for South cities:

In [18]:
water[water['location'] == 'South'].corr()

,mortality,hardness
mortality,1.000000,-0.602153
hardness,-0.602153,1.000000


Среди респондентов General Social Survey 2014 года хотя бы раз в месяц проводят вечер в баре 203 женщины и 239 мужчин; реже, чем раз в месяц, это делают 718 женщин и 515 мужчин.  

Посчитайте значение коэффициента корреляции Мэтьюса между полом и частотой похода в бары. Округлите значение до трёх знаков после десятичной точки.

Matthews correlation:

$MCC_{X_1X_2} = \frac{ad - bc}{(a + b)(a + c)(b + d)(c + d)}$

In [3]:
a = 718
b = 203
c = 515
d = 239

In [4]:
MCC = (a*d - b*c)/np.sqrt((a+b)*(a + c)*(b + d)*(c + d))

In [5]:
MCC

0.10900237458678963

In [12]:
import scipy.stats as stats

In [30]:
table = np.array([[a,b],[c,d]])

In [31]:
table

array([[718, 203],
       [515, 239]])

In [32]:
scipy.stats.chi2_contingency(table)

(19.40753078854304,
 1.0558987006638725e-05,
 1,
 array([[677.96597015, 243.03402985],
        [555.03402985, 198.96597015]]))

В предыдущей задаче давайте попробуем ответить на немного другой вопрос: отличаются ли доля мужчин и доля женщин, относительно часто проводящих вечера в баре? Постройте 95% доверительный интервал для разности долей, вычитая долю женщин из доли мужчин. Чему равна его нижняя граница? Округлите до четырёх знаков после десятичной точки.

In [13]:
def proportions_diff_confint_ind_from_values(s1, n1, s2, n2, alpha=0.05):
    z = stats.norm.ppf(1 - alpha / 2.)
    
    p1 = s1 / float(n1)
    p2 = s2 / float(n2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)
    
    return (left_boundary, right_boundary)

def proportions_diff_confint_ind(sample1, sample2, alpha=0.05):    
    s1 = sum(sample1)
    s2 = sum(sample2)
    n1 = len(sample1)
    n2 = len(sample2)
    
    return proportions_diff_confint_ind_from_values(s1, n1, s2, n2, alpha=alpha)

In [14]:
s1 = d
n1 = c + d
s2 = b
n2 = a + b

In [15]:
conf_int = proportions_diff_confint_ind_from_values(s1, n1, s2, n2)

In [16]:
conf_int

(0.053905233215813156, 0.13922183141523897)

Проверьте гипотезу о равенстве долей любителей часто проводить вечера в баре среди мужчин и женщин. Посчитайте достигаемый уровень значимости, используя двустороннюю альтернативу. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 
−8
 , нужно ввести 8).

In [17]:
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [18]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [19]:
def proportions_diff_z_stat_ind_from_values(s1, n1, s2, n2):
    p1 = s1 / n1
    p2 = s2 / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [21]:
p_val = proportions_diff_z_test(proportions_diff_z_stat_ind_from_values(s1,n1,s2,n2))

In [22]:
p_val

8.153453089576601e-06

Посмотрим на данные General Social Survey 2014 года и проанализируем, как связаны ответы на вопросы "Счастливы ли вы?" и "Довольны ли вы вашим финансовым положением?"
Чему равно значение статистики хи-квадрат для этой таблицы сопряжённости? Округлите ответ до четырёх знаков после десятичной точки.

In [23]:
happiness_job_table = np.array([[197,111,33],[382,685,331],[110,342,333]])

In [27]:
chi_2 = stats.chi2_contingency(happiness_job_table)[0]
chi_2

293.68311039689746

Чему в предыдущей задаче равно значение коэффициента V Крамера для рассматриваемых признаков? Округлите ответ до четырёх знаков после десятичной точки. 

$\phi_c(X_1^n, X_2^n) = \sqrt{\frac{\chi^2(X_1^n, X_2^n)}{n(min(K_1,K_2) - 1)}}$

In [31]:
n = sum(sum(happiness_job_table))
k_min = 3
n

2524

In [32]:
fi = np.sqrt(chi_2/(n*(k_min - 1)))

In [33]:
fi

0.2412013934500338